# Preprocessing and Feature Extraction

In this notebook, we are applying preprocessing steps and extracting features from our dataset. 

Firstly, we remove outlying points using the first derivation of the power consumption curve (data points with significant sudden change). Then we use the Prophet by Facebook [[1]] to extract seasonalities and trends from our data.


[1]: https://facebook.github.io/prophet/


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import csv
from pathlib import Path

import holidays
import numpy as np
import pandas as pd
from fbprophet import Prophet
from fbprophet.models import PyStanBackend
from tqdm.notebook import tqdm

In [ ]:
LOAD_PATH = Path("../electric_grid_data/")
SAVE_PATH = Path("../power_consumption_data")

In [ ]:
# Creating Pandas indices for our time series
def create_weekly_index(seasson, header):
    tmp = pd.Series(seasson, index=header).resample("7d").sum()

    start_seasson = tmp[tmp == 24 * 7].index[0]

    return pd.date_range(start=start_seasson, freq="1h", periods=24 * 7)


def create_daily_index(day, header):
    tmp = pd.Series(day, index=header).resample("1d").sum()

    start_day = tmp[tmp == 24].index[0]

    return pd.date_range(start=start_day, freq="1h", periods=24)

In [ ]:
# The older version of the Prophet did not allow
# skipping Newton optimization, which is slower and
# takes too much time on our dataset.
# This workaround should not be necessary once
# you're using newer versions of Prophet.
class BackendWithoutNewton(PyStanBackend):
    def fit(self, stan_init, stan_data, **kwargs) -> dict:

        args = dict(
            data=stan_data,
            init=lambda: stan_init,
            algorithm="Newton" if stan_data["T"] < 100 else "LBFGS",
            iter=1e4,
        )
        args.update(kwargs)

        # Remove Newton fallback
        self.stan_fit = self.model.optimizing(**args)

        params = dict()

        for par in self.stan_fit.keys():
            params[par] = self.stan_fit[par].reshape((1, -1))

        return params

In [ ]:
with open(LOAD_PATH.joinpath("data_grid.csv"), newline="") as input_file, open(
    SAVE_PATH.joinpath("kwh_hours.csv"), "w"
) as kwh_hours_file, open(
    SAVE_PATH.joinpath("work_day.csv"), "w"
) as work_day_file, open(
    SAVE_PATH.joinpath("free_day.csv"), "w"
) as free_day_file, open(
    SAVE_PATH.joinpath("spring_week.csv"), "w"
) as spring_week_file, open(
    SAVE_PATH.joinpath("summer_week.csv"), "w"
) as summer_week_file, open(
    SAVE_PATH.joinpath("autumn_week.csv"), "w"
) as autumn_week_file, open(
    SAVE_PATH.joinpath("winter_week.csv"), "w"
) as winter_week_file, open(
    SAVE_PATH.joinpath("trend.csv"), "w"
) as trend_file, open(
    SAVE_PATH.joinpath("year.csv"), "w"
) as year_file, open(
    SAVE_PATH.joinpath("residuals_hist.csv"), "w"
) as res_hist_file, open(
    SAVE_PATH.joinpath("residuals_bin.csv"), "w"
) as res_bin_file:

    reader = csv.reader(input_file, delimiter=";")
    w_kwh_hours = csv.writer(kwh_hours_file, delimiter=",")

    w_work_day = csv.writer(work_day_file, delimiter=",")
    w_free_day = csv.writer(free_day_file, delimiter=",")

    w_spring_week = csv.writer(spring_week_file, delimiter=",")
    w_summer_week = csv.writer(summer_week_file, delimiter=",")
    w_autumn_week = csv.writer(autumn_week_file, delimiter=",")
    w_winter_week = csv.writer(winter_week_file, delimiter=",")

    w_trend = csv.writer(trend_file, delimiter=",")
    w_year = csv.writer(year_file, delimiter=",")

    w_rhist = csv.writer(res_hist_file, delimiter=",")
    w_rbin = csv.writer(res_bin_file, delimiter=",")

    for index, line in enumerate(tqdm(reader)):
        if index == 0:
            # Header row with dates
            header = pd.to_datetime(line[4:])

            s_header = (
                pd.Series(np.ones(len(header)), index=header).resample("1h").mean()
            )
            s_header = s_header[~np.isnan(s_header)]
            s_index = s_header.index

            dates_frame = pd.DataFrame(index=s_index)
            dates_frame["ds"] = s_index
            dates_frame["y"] = np.nan

            dates_frame["spring"] = np.isin(s_index.month, [3, 4, 5])
            dates_frame["summer"] = np.isin(s_index.month, [6, 7, 8])
            dates_frame["autumn"] = np.isin(s_index.month, [9, 10, 11])
            dates_frame["winter"] = np.isin(s_index.month, [12, 1, 2])

            # initializing holydays
            sk_h = holidays.Slovakia()

            dates_frame["free_day"] = np.array([x in sk_h for x in s_index]) | np.isin(
                s_index.day_name(), ["Saturday", "Sunday"]
            )
            dates_frame["work_day"] = ~dates_frame["free_day"]

            spring_index = create_weekly_index(dates_frame["spring"], s_index)
            summer_index = create_weekly_index(dates_frame["summer"], s_index)
            autumn_index = create_weekly_index(dates_frame["autumn"], s_index)
            winter_index = create_weekly_index(dates_frame["winter"], s_index)

            free_day_index = create_daily_index(dates_frame["free_day"], s_index)
            work_day_index = create_daily_index(dates_frame["work_day"], s_index)

            w_kwh_hours.writerow(s_index)

            continue

        if line[2].lower() != "p+":
            continue

        # Prepare data
        try:
            data = np.array(line[4:])
            data[data == ""] = np.nan  # setting empty cells to nan
            data = data.astype(float)
            data[data < 0] = np.nan  # removing values bellow zero
        except Exception as e:
            print(e)
            continue

        data = pd.Series(data, index=header).resample("15min").mean()

        # Removing anomalous points by using the first derivation
        # of the power consumption curve
        s_diff = data.diff().diff()

        q_low = np.nanquantile(s_diff, 0.005)
        q_high = np.nanquantile(s_diff, 0.995)

        data = data[(s_diff > q_low) & (s_diff < q_high)]

        data = data.resample("1h").mean() * 4
        data = data[~np.isnan(data)]

        # Remove data with a lot of nans
        if data.shape[0] / s_index.shape[0] < 0.5:
            continue

        # Prepare data for Prophet

        dates_frame.loc[data.index, "y"] = np.log1p(data)

        # Setup Prophet
        # Prophet definition
        m = Prophet(
            n_changepoints=0,
            growth="linear",
            seasonality_mode="additive",
            yearly_seasonality=False,
            weekly_seasonality=False,
            daily_seasonality=False,
        )
        m.stan_backend = BackendWithoutNewton()

        # Setting-up seassonalities and holidays in Slovakia
        m.add_country_holidays(country_name="SK")

        m.add_seasonality(name="yearly", period=365, fourier_order=20)
        m.add_seasonality(
            name="weekly_spring", period=7, fourier_order=5, condition_name="spring"
        )
        m.add_seasonality(
            name="weekly_summer", period=7, fourier_order=5, condition_name="summer"
        )
        m.add_seasonality(
            name="weekly_autumn", period=7, fourier_order=5, condition_name="autumn"
        )
        m.add_seasonality(
            name="weekly_winter", period=7, fourier_order=5, condition_name="winter"
        )

        m.add_seasonality(
            name="work_day", period=1, fourier_order=3, condition_name="work_day"
        )
        m.add_seasonality(
            name="free_day", period=1, fourier_order=3, condition_name="free_day"
        )

        # Fit & predict Prophet
        try:
            m.fit(dates_frame)
            forecast = m.predict(dates_frame)
        except Exception as e:
            print(e)
            continue

        df = pd.merge(dates_frame[["ds", "y"]], forecast, on="ds")
        df.set_index("ds", inplace=True)

        free_day_data = df["free_day"].loc[free_day_index]
        work_day_data = df["work_day"].loc[work_day_index]

        spring_data = df["weekly_spring"].loc[spring_index].resample("2h").mean()
        summer_data = df["weekly_summer"].loc[summer_index].resample("2h").mean()
        autumn_data = df["weekly_autumn"].loc[autumn_index].resample("2h").mean()
        winter_data = df["weekly_winter"].loc[winter_index].resample("2h").mean()

        trend_data = df["trend"].resample("w").mean().interpolate()
        year_data = df["yearly"].resample("3d").mean().interpolate()

        residuals = df["yhat"] - df["y"]
        hist, bin_edges = np.histogram(residuals, "auto", density=True)

        # Write data
        w_kwh_hours.writerow(np.expm1(df["y"]).round(3))

        w_free_day.writerow(free_day_data)
        w_work_day.writerow(work_day_data)

        w_spring_week.writerow(spring_data)
        w_summer_week.writerow(summer_data)
        w_autumn_week.writerow(autumn_data)
        w_winter_week.writerow(winter_data)

        w_trend.writerow(trend_data)
        w_year.writerow(year_data)

        w_rhist.writerow(hist)
        w_rbin.writerow(bin_edges)